In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

#뉴스 링크 불러오기
f = open('C:/Users/LG/OneDrive/바탕 화면/코딩/뉴스 링크(comment 링크 추가).txt', 'r')
links = f.read().split('https://')

news_link = []
for link in links:
    if len(link) > 0:
        link = link.replace('\t', '').replace('\n', '')
        if 'daum' in link:
            news_link.append("https://"+link)
        else:
            news_link.append("https://"+link+'&m_view=1')

#댓글 데이터프레임 모양
commentdf = pd.DataFrame({'comment' : [], 'news_title':[]})

#크롤링 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.20 Safari/537.36")

#크롤링 코드
for url in news_link:
    if 'naver' in url: 
        #웹 드라이버
        driver = webdriver.Chrome('./chromedriver.exe', options=options)
        driver.implicitly_wait(30)
        driver.get(url)

        #클린봇 옵션 해제
        time.sleep(1)
        cleanbot = driver.find_element(By.CSS_SELECTOR, 'a.u_cbox_cleanbot_setbutton')
        cleanbot.click()
        time.sleep(1)
        cleanbot_disable = driver.find_element(By.XPATH, "//input[@id='cleanbot_dialog_checkbox_cbox_module']")
        cleanbot_disable.click()
        time.sleep(1)
        cleanbot_confirm = driver.find_element(By.CSS_SELECTOR, 'body > div.u_cbox.u_cbox_layer_wrap > div > div.u_cbox_layer_cleanbot2 > div.u_cbox_layer_cleanbot2_extra > button')
        cleanbot_confirm.click()
        time.sleep(1)

        #댓글 더보기 계속 클릭하기
        while True:
            try:
                btn_more = driver.find_element(By.CSS_SELECTOR, 'a.u_cbox_btn_more')
                btn_more.click()
                time.sleep(1)
            except:
                break

        #댓글추출
        contents = driver.find_elements(By.CSS_SELECTOR, 'span.u_cbox_contents')
        #뉴스 제목 추출
        news_title = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_title > h2')

        for content in contents:
            new_row = pd.DataFrame([[content.text, news_title.text]], columns = commentdf.columns)
            commentdf = pd.concat([commentdf, new_row])
        driver.quit()

    elif 'daum' in url: 
        #웹 드라이버
        driver = webdriver.Chrome('./chromedriver.exe', options=options)
        driver.implicitly_wait(30)
        driver.get(url)

        time.sleep(30)
        reply_open = driver.find_element(By.CSS_SELECTOR, '#alex-area > div > div > div > div.cmt_count_box > button')
        reply_open.click()
        time.sleep(1)

        #최신순
        safebot = driver.find_element(By.XPATH, '//*[@id="alex-area"]/div/div/div/div[4]/ul[1]/li[3]/button')
        safebot.click()
        time.sleep(1)

        #세이프봇 옵션 해제
        safebot = driver.find_element(By.XPATH, '//*[@id="alex-area"]/div/div/div/div[3]/div[2]/button')
        safebot.click()
        time.sleep(1)
        safebot_disable = driver.find_element(By.CSS_SELECTOR, '#alex-area > div > div > div:nth-child(2) > div.cmtarray_layer > div > div.layer_body > dl > dd > button')
        safebot_disable.click()
        time.sleep(1)
        safebot_confirm = driver.find_element(By.XPATH, '//*[@id="alex-area"]/div/div/div[2]/div[2]/div/a')
        safebot_confirm.click()
        time.sleep(1)


        #더보기 계속 클릭하기
        while True:
            try:
                btn_more = driver.find_element(By.CSS_SELECTOR, '#alex-area > div > div > div > div.cmt_box > div.alex_more > button')
                btn_more.click()
                time.sleep(1)
            except:
                break


        #댓글추출
        contents = driver.find_elements(By.CSS_SELECTOR, '#alex-area > div > div > div > div.cmt_box > ul.list_comment > li > div > p')

        #제목추출
        news_title = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.head_view > h3')

        for content in contents:
            content = content.text
            content = content.replace("\n", "")
            new_row = pd.DataFrame([[content, news_title.text]], columns = commentdf.columns)
            commentdf = pd.concat([commentdf, new_row])
        driver.quit()

C:\Users\LG\AppData\Local\Temp\ipykernel_15804\2090645560.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', options=options)
C:\Users\LG\AppData\Local\Temp\ipykernel_15804\2090645560.py:68: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', options=options)


In [7]:
commentdf

,comment,news_title
0,아직도느낌안오나?,"윤호중·이재명과 손맞잡더니…박지현 또 ""민주당 바꾸고싶다"""
0,이게 ㅈㆍㄴ 웃긴게 더듬고만진당 바꾼다면서 찢죄명이랑 윤호중 손을잡는데ㅋㅋㅋ 이게 ...,"윤호중·이재명과 손맞잡더니…박지현 또 ""민주당 바꾸고싶다"""
0,내일이면 결과나온다 이번지방선거도 이겨야 정신차릴거다 모르지 국회의원 선거때까지도 ...,"윤호중·이재명과 손맞잡더니…박지현 또 ""민주당 바꾸고싶다"""
0,박지현아 추미애한테 전화해봐라.좋은 말씀 해줄꺼다.ㅋㅋㅋ.추미애뇬은 윤석렬대통령당선...,"윤호중·이재명과 손맞잡더니…박지현 또 ""민주당 바꾸고싶다"""
0,ㅋㅋㅋㅋ 본인들도 진심이 아니니 어색하죠?! 보는 사람들한테도 그게 느껴짐,"윤호중·이재명과 손맞잡더니…박지현 또 ""민주당 바꾸고싶다"""
...,...,...
0,관상은 과학이라더니 욕심 가득한 얼굴,"윤호중, 배우출신 與후보에 ""일흔 넘어 새 것 하기엔 그렇지 않나""..연령차별 부적절"
0,맞는 말이네,"윤호중, 배우출신 與후보에 ""일흔 넘어 새 것 하기엔 그렇지 않나""..연령차별 부적절"
0,윤호중후보님댁들은 정치감 떨어졌으니그만들 두시죠,"윤호중, 배우출신 與후보에 ""일흔 넘어 새 것 하기엔 그렇지 않나""..연령차별 부적절"
0,그 정도 디스는 애교 아니냐? 뉴시스 또 시작이네.,"윤호중, 배우출신 與후보에 ""일흔 넘어 새 것 하기엔 그렇지 않나""..연령차별 부적절"


In [13]:
# csv 파일로 저장
commentdf.to_csv("comment_total.csv", encoding='utf-8-sig')

In [ ]:
# 분할 추출 방법
# 위에서 뉴스 링크를 [0:20] 정도로 나누어 추출하고,
# 추출된 commentdf를 df(i) 형식으로 저장한 뒤,
# commnet_total.csv 파일 생성

df_list = []

#외부에 저장된
for i in range(1, 11):
  globals()['df{}'.format(i)] = pd.read_csv("./df{}.csv".format(i))
  df_list.append(globals()['df{}'.format(i)])

commentdf = pd.DataFrame({'comment' : [], 'news_title':[]})

for df in df_list:
  df_copy = df
  commentdf = pd.concat([commentdf, df_copy])

commentdf.to_csv("comment_total.csv", encoding='utf-8-sig')